### IMPORTS

In [454]:
#imports
import pandas as pd
import numpy as np
import matplotlib as mp
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### DATASET

In [455]:
# DATASET = https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

In [456]:
movies = pd.read_csv('/content/sample_data/tmdb_5000_movies.csv')

### PRE PROCESSING

In [457]:
movies.head()#Checking data frame

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [458]:
movies.info()#Checking info of dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [459]:
movies.shape#checking rows and columns of dataframe

(4803, 20)

In [460]:
ds = movies[0:400]#Using first 400 rows
print(ds.columns)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')


In [461]:
ds.dropna(inplace = True) # Remove Null values, Not necessary since no null values as seen above
ds

<ipython-input-461-2298d20a5e0f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds.dropna(inplace = True) # Remove Null values, Not necessary since no null values as seen above


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,75000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",http://www.towerheist.com/index.php,59108,"[{""id"": 1418, ""name"": ""skyscraper""}, {""id"": 45...",en,Tower Heist,A luxury condo manager leads a staff of worker...,36.743324,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2011-11-02,152930623,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Ordinary guys. An extraordinary robbery.,Tower Heist,5.8,932
395,85000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",http://www.sonypictures.com/movies/theholiday/,1581,"[{""id"": 65, ""name"": ""holiday""}, {""id"": 212, ""n...",en,The Holiday,"Two women, one (Cameron Diaz) from America and...",44.967453,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2006-12-08,194168700,136.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,It's Christmas Eve and we are going to go cele...,The Holiday,6.7,1225
397,850

In [462]:
new_ds = ds[['genres','keywords', 'overview','title','original_title']] # using only necessary columns

In [463]:
new_ds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260 entries, 0 to 399
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   genres          260 non-null    object
 1   keywords        260 non-null    object
 2   overview        260 non-null    object
 3   title           260 non-null    object
 4   original_title  260 non-null    object
dtypes: object(5)
memory usage: 20.3+ KB


In [464]:
new_ds

,genres,keywords,overview,title,original_title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,Avatar
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,Spectre
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,The Dark Knight Rises
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,John Carter
...,...,...,...,...,...
394,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...","[{""id"": 1418, ""name"": ""skyscraper""}, {""id"": 45...",A luxury condo manager leads a staff of worker...,Tower Heist,Tower Heist
395,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...","[{""id"": 65, ""name"": ""holiday""}, {""id"": 212, ""n...","Two women, one (Cameron Diaz) from America and...",The Holiday,The Holiday
397,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...","[{""id"": 3687, ""name"": ""graduation""}, {""id"": 59...","Ten years after their divorce, Jane and Jake A...",It's Complicated,It's Complicated
398,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name...","[{""id"": 585, ""name"": ""casino""}, {""id"": 9727, ""...",Danny Ocean's team of criminals are back and c...,Ocean's Thirteen,Ocean's Thirteen


In [465]:
new_ds.isnull().sum()# checking for missing values

,0
genres,0
keywords,0
overview,0
title,0
original_title,0


In [466]:
new_ds.duplicated().sum()#Checking for duplicated values

0

In [467]:
new_ds.iloc[0].genres
print(new_ds.iloc[0].genres)
print(new_ds.iloc[0].keywords)

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]


In [468]:
def convert(obj): #Function to convert to a list
    l = []
    for i in ast.literal_eval(obj) :
        l.append(i['name'])
    return l

In [469]:
new_ds['genres'] = new_ds['genres'].apply(convert) #Converting to a list

<ipython-input-469-d3458ca5e489>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['genres'] = new_ds['genres'].apply(convert) #Converting to a list


In [470]:
new_ds['keywords'] = new_ds['keywords'].apply(convert) #Converting to a list

<ipython-input-470-9a82a169a6f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['keywords'] = new_ds['keywords'].apply(convert) #Converting to a list


In [471]:
new_ds

,genres,keywords,overview,title,original_title
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar,Avatar
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre,Spectre
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter,John Carter
...,...,...,...,...,...
394,"[Action, Comedy]","[skyscraper, thanksgiving, billionaire, parade...",A luxury condo manager leads a staff of worker...,Tower Heist,Tower Heist
395,"[Comedy, Romance]","[holiday, london england, film making, christm...","Two women, one (Cameron Diaz) from America and...",The Holiday,The Holiday
397,"[Comedy, Romance]","[graduation, ex husband, woman director]","Ten years after their divorce, Jane and Jake A...",It's Complicated,It's Complicated
398,"[Crime, Thriller]","[casino, thief, revenge, heist, las vegas, pre...",Danny Ocean's team of criminals are back and c...,Ocean's Thirteen,Ocean's Thirteen


In [472]:
mismatched_indices = new_ds.index[new_ds['title'] != new_ds['original_title']].tolist()
print("\nIndices of mismatches:", mismatched_indices) # checking if there is a difference between original title and title


Indices of mismatches: []


In [473]:
new_ds.drop(['original_title'],axis=1,inplace=True) #Orginal_title and Title were the same

<ipython-input-473-eda6b1a7cd5f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds.drop(['original_title'],axis=1,inplace=True) #Orginal_title and Title were the same


In [474]:
new_ds.head()

,genres,keywords,overview,title
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Avatar
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Spectre
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,The Dark Knight Rises
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",John Carter


In [475]:
new_ds['keywords'] = new_ds['keywords'].apply(lambda x:" ".join(x)) #List to string


new_ds['overview'].apply(lambda x:x.lower()) #Conversion to lower case
new_ds['keywords'].apply(lambda x:x.lower()) #Conversion to lower case

new_ds['genres'] = new_ds['genres'].apply(lambda x: x.split() if isinstance(x, str) else x) #List to string
new_ds['genres'] = new_ds['genres'].apply(lambda x: [genre.lower() for genre in x] if isinstance(x, list) else x) #List to string




<ipython-input-475-e54c6ee185ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['keywords'] = new_ds['keywords'].apply(lambda x:" ".join(x)) #List to string
<ipython-input-475-e54c6ee185ba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['genres'] = new_ds['genres'].apply(lambda x: x.split() if isinstance(x, str) else x) #List to string
<ipython-input-475-e54c6ee185ba>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [476]:
new_ds['genres'] = new_ds['genres'].apply(lambda x:" ".join(x)) #List to string

<ipython-input-476-e588c17c1ad7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['genres'] = new_ds['genres'].apply(lambda x:" ".join(x)) #List to string


In [477]:
#new_ds['genres'][0]

In [478]:
new_ds['genres'] = new_ds['genres'].apply(lambda x: x.strip())#Getting rid of extra spaces
new_ds['keywords'] = new_ds['keywords'].apply(lambda x: x.strip())#Getting rid of extra spaces
new_ds['overview'] = new_ds['overview'].apply(lambda x: x.strip())#Getting rid of extra spaces
new_ds['title'] = new_ds['title'].apply(lambda x: x.strip())#Getting rid of extra spaces

<ipython-input-478-3cdf77867cea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['genres'] = new_ds['genres'].apply(lambda x: x.strip())#Getting rid of extra spaces
<ipython-input-478-3cdf77867cea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds['keywords'] = new_ds['keywords'].apply(lambda x: x.strip())#Getting rid of extra spaces
<ipython-input-478-3cdf77867cea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [479]:
new_ds.head()

,genres,keywords,overview,title
0,action adventure fantasy science fiction,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",Avatar
1,adventure fantasy action,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End
2,action adventure crime,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,Spectre
3,action crime drama thriller,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,The Dark Knight Rises
4,action adventure science fiction,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",John Carter


### TRANSFORMATION

In [480]:
# Function to fit TF-IDF vectorizers
def fit_vectorizers(data):
    """Fits TF-IDF vectorizers for keywords and overview."""
    vectorizer_keywords = TfidfVectorizer(stop_words='english')
    tfidf_keywords = vectorizer_keywords.fit_transform(data['keywords'])

    vectorizer_overview = TfidfVectorizer(stop_words='english')
    tfidf_overview = vectorizer_overview.fit_transform(data['overview'])

    return (vectorizer_keywords, vectorizer_overview), (tfidf_keywords, tfidf_overview)

In [481]:
# Function to transform query
def transform_query(query, vectorizers):
    """Transforms the query using the same TF-IDF vectorizers."""
    vectorizer_keywords, vectorizer_overview = vectorizers
    query_vec_keywords = vectorizer_keywords.transform([query])
    query_vec_overview = vectorizer_overview.transform([query])
    return query_vec_keywords, query_vec_overview


In [482]:
# Function to compute similarity
def compute_similarity(query_vec_keywords, query_vec_overview, tfidf_keywords, tfidf_overview):
    """Computes cosine similarity between query and dataset."""
    cosine_sim_keywords = cosine_similarity(query_vec_keywords, tfidf_keywords).flatten()
    cosine_sim_overview = cosine_similarity(query_vec_overview, tfidf_overview).flatten()
    similarity_score = (cosine_sim_keywords + cosine_sim_overview) / 2  # Averaging both
    return similarity_score


In [483]:
def recommend(data, query, vectorizers, tfidf_matrices, top_n=5):
    """Returns top N recommendations based on cosine similarity."""
    query_vec_keywords, query_vec_overview = transform_query(query, vectorizers)
    tfidf_keywords, tfidf_overview = tfidf_matrices
    similarity_scores = compute_similarity(query_vec_keywords, query_vec_overview, tfidf_keywords, tfidf_overview)

    # Add similarity scores to the dataset
    data["similarity"] = similarity_scores

    # Sort and get top recommendations
    recommendations = data.sort_values(by="similarity", ascending=False).head(top_n)[["title"]]
    print(recommendations)

# Example usage
vectorizers, tfidf_matrices = fit_vectorizers(new_ds)
recommend(new_ds, "I like car movies", vectorizers, tfidf_matrices)

<ipython-input-483-a71544e2dc43>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"] = similarity_scores


              title
44        Furious 7
40           Cars 2
227  Knight and Day
204       Fast Five
225     Speed Racer


In [488]:
def recommend(data, query, vectorizers, tfidf_matrices, top_n=5):
    """Returns top N recommendations based on cosine similarity."""
    query_vec_keywords, query_vec_overview = transform_query(query, vectorizers)
    tfidf_keywords, tfidf_overview = tfidf_matrices
    similarity_scores = compute_similarity(query_vec_keywords, query_vec_overview, tfidf_keywords, tfidf_overview)

    # Add similarity scores to the dataset
    data["similarity"] = similarity_scores

    # Sort and get top recommendations
    recommendations = data.sort_values(by="similarity", ascending=False).head(top_n)[["title"]]
    print(recommendations)

# Example usage
vectorizers, tfidf_matrices = fit_vectorizers(new_ds)
recommend(new_ds, "I love fighting movies", vectorizers, tfidf_matrices)

                              title
226                 How Do You Know
119                   Batman Begins
257                      Real Steel
115                         Hancock
211  How the Grinch Stole Christmas


<ipython-input-488-19320dd01c5e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"] = similarity_scores


In [484]:
# Recommendation function
def recommend(data, query, vectorizers, tfidf_matrices, top_n=5):
    """Returns top N recommendations based on cosine similarity."""
    query_vec_keywords, query_vec_overview = transform_query(query, vectorizers)
    tfidf_keywords, tfidf_overview = tfidf_matrices
    similarity_scores = compute_similarity(query_vec_keywords, query_vec_overview, tfidf_keywords, tfidf_overview)

    # Create a copy to prevent modifying the original DataFrame
    data_copy = data.copy()
    data_copy["similarity"] = similarity_scores

    # Sort and get top recommendations
    recommendations = data_copy.sort_values(by="similarity", ascending=False).head(top_n)[["title"]]
    print(recommendations)


# Example usage
vectorizers, tfidf_matrices = fit_vectorizers(new_ds)
recommend(new_ds, "I love thrilling action movies set in space, with a comedic twist.", vectorizers, tfidf_matrices)


                       title
239                  Gravity
0                     Avatar
226          How Do You Know
95              Interstellar
94   Guardians of the Galaxy


### TESTING

In [486]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Vectorizer for keywords
vectorizer_keywords = TfidfVectorizer(stop_words='english')
tfidf_keywords = vectorizer_keywords.fit_transform(new_ds['keywords'])

# Vectorizer for overview
vectorizer_overview = TfidfVectorizer(stop_words='english')
tfidf_overview = vectorizer_overview.fit_transform(new_ds['overview'])

# Vectorizer for genres
vectorizer_genres = TfidfVectorizer(stop_words='english')
tfidf_genres = vectorizer_genres.fit_transform(new_ds['genres'])

def recommend2(query):
    # Transform query using the same vectorizers
    query_vec1 = vectorizer_keywords.transform([query])  # Match `keywords` vectorizer
    query_vec2 = vectorizer_overview.transform([query])  # Match `overview` vectorizer
    query_vec3 = vectorizer_genres.transform([query])  # Match `genres` vectorizer

    # Compute cosine similarity
    cosine_sim1 = cosine_similarity(query_vec1, tfidf_keywords).flatten()
    cosine_sim2 = cosine_similarity(query_vec2, tfidf_overview).flatten()
    cosine_sim3 = cosine_similarity(query_vec3, tfidf_genres).flatten()

    # Take the average similarity from both
    new_ds["similarity1"] = (cosine_sim1 + cosine_sim2) / 2
    new_ds["similarity2"] = cosine_sim1
    new_ds["similarity3"] = cosine_sim2
    new_ds["similarity4"] = (cosine_sim1 + cosine_sim3) / 2


    df_sorted1 = new_ds.sort_values(by="similarity1", ascending=False)
    top_n = 5
    recommendations1 = df_sorted1.head(top_n)[["title", "similarity1"]]

    df_sorted2 = new_ds.sort_values(by="similarity2", ascending=False)
    top_n = 5
    recommendations2 = df_sorted2.head(top_n)[["title", "similarity2"]]

    df_sorted3 = new_ds.sort_values(by="similarity3", ascending=False)
    top_n = 5
    recommendations3 = df_sorted3.head(top_n)[["title", "similarity3"]]

    df_sorted4 = new_ds.sort_values(by="similarity4", ascending=False)
    top_n = 5
    recommendations4 = df_sorted4.head(top_n)[["title", "similarity4"]]


    print(recommendations1)
    print(recommendations2)
    print(recommendations3)
    print(recommendations4)


    #return recommendations1, recommendations2, recommendations3

# Running the query
print(recommend2("I like car movies"))


              title  similarity1
44        Furious 7     0.339066
40           Cars 2     0.257522
227  Knight and Day     0.182321
204       Fast Five     0.170675
225     Speed Racer     0.163628
              title  similarity2
44        Furious 7     0.678131
227  Knight and Day     0.364641
40           Cars 2     0.342342
204       Fast Five     0.341349
225     Speed Racer     0.327255
                                  title  similarity3
40                               Cars 2     0.172703
323                  Sex and the City 2     0.082533
86                  Shrek Forever After     0.080422
271                          The Island     0.079503
9    Batman v Superman: Dawn of Justice     0.078821
              title  similarity4
44        Furious 7     0.339066
227  Knight and Day     0.182321
40           Cars 2     0.171171
204       Fast Five     0.170675
225     Speed Racer     0.163628
None


<ipython-input-486-65189a6d5755>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds["similarity1"] = (cosine_sim1 + cosine_sim2) / 2
<ipython-input-486-65189a6d5755>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_ds["similarity2"] = cosine_sim1
<ipython-input-486-65189a6d5755>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [487]:
# SALARY EXPECTATIONS
# 2800$-4200$ per month